In [ ]:
token = "<insert_github_token_here>"

In [ ]:
import requests

# Make a GET request to get repository information
url = 'https://api.github.com/repos/ronaldozica/Projetos-JS'
headers = {'Authorization': f'token {token}'}

response = requests.get(url, headers=headers)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()
    print(data)
    # Use the data as needed
else:
    print(f"Request failed with status code {response.status_code}: {response.text}")

In [ ]:
import requests
from datetime import datetime

# Define the date as 'YYYY-MM-DD'
since_date = '2023-01-01'
formatted_since_date = datetime.strptime(since_date, '%Y-%m-%d').date().isoformat()

# Set up the search parameters
search_query = f'machine learning in:title stars:>50 pushed:>{formatted_since_date} sort:stars-desc is:issue'
url = 'https://api.github.com/search/repositories'
headers = {'Authorization': f'token {token}'}
params = {'q': search_query}

# Make a GET request to search for repositories
response = requests.get(url, headers=headers, params=params)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()
    # Extract the list of repositories from the response
    repositories = data['items']
    
    total_matches = data['total_count']
    print(f"Total Matches: {total_matches}")
    print("-------------")

    for repo in repositories:
        print(f"Repository Name: {repo['name']}")
        print(f"Stars: {repo['stargazers_count']}")
        print(f"Url: {repo['html_url']}")
        print("-------------")

else:
    print(f"Request failed with status code {response.status_code}: {response.text}")


In [ ]:
import requests
from datetime import datetime

page_size = 100  # Set to the maximum value allowed by GitHub API

# Define the date as 'YYYY-MM-DD'
since_date = '2023-01-01'
formatted_since_date = datetime.strptime(since_date, '%Y-%m-%d').date().isoformat()

# Set up the search parameters
search_query = f'machine learning in:title stars:>50 pushed:>{formatted_since_date} sort:stars-desc is:issue'
url = 'https://api.github.com/search/repositories'
headers = {'Authorization': f'token {token}'}

# Make the initial request to get the total number of repositories
params = {'q': search_query, 'per_page': 1, 'page': 1}
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    total_matches = data['total_count']
    print(f"Total Matches: {total_matches}")
    print("-------------")

    # Calculate the number of pages needed for pagination
    total_pages = (total_matches // page_size) + 1

    # Save information for each page
    with open('repositories_info.txt', 'w') as file:
        for page in range(1, total_pages + 1):
            params = {'q': search_query, 'per_page': page_size, 'page': page}
            response = requests.get(url, headers=headers, params=params)

            if response.status_code == 200:
                data = response.json()
                repositories = data['items']

                for repo in repositories:
                    repo_name = repo['name']
                    stars_count = repo['stargazers_count']
                    repo_url = repo['html_url']

                    # Write repository information to the file
                    file.write(f"Repository Name: {repo_name}\n")
                    file.write(f"Stars: {stars_count}\n")
                    file.write(f"Url: {repo_url}\n")
                    file.write("-------------\n")

                    # Store additional information for making a GET request to the repository
                    file.write(f"Clone URL: {repo['clone_url']}\n")
                    file.write(f"Default Branch: {repo['default_branch']}\n")
                    file.write(f"Owner: {repo['owner']['login']}\n")
                    file.write("===================================\n")

            else:
                print(f"Request failed with status code {response.status_code}: {response.text}")

    print("Information saved to 'repositories_info.txt'")
else:
    print(f"Request failed with status code {response.status_code}: {response.text}")


In [ ]:
import requests
import base64

# Read repository information from file
with open('repositories_info.txt', 'r') as file:
    repo_info = file.read()

# Split repository information into individual repositories
repos = repo_info.split('===================================')

# Remove empty strings from the list
repos = [repo.strip() for repo in repos if repo.strip()]

# GitHub API endpoint for retrieving README content
readme_endpoint = 'https://api.github.com/repos/{owner_and_repo}/readme'

# GitHub API headers
headers = {'Authorization': f'token {token}'}

# Keywords to check for in the README content
keywords_to_check = ['demo', 'lesson', 'quiz', 'study', 'student']

# Output file
output_file_path = 'output.txt'

with open(output_file_path, 'w') as output_file:
    for repo in repos:
        # Extract repository information
        repo_name = repo.split('\n')[0].split(': ')[1].strip()
        owner = repo.split('\n')[-2].split(': ')[1].strip()
        clone_url = repo.split('\n')[4].split(': ')[1].strip()

        # Remove ".git" extension from clone URL
        clone_url = clone_url.replace('.git', '')

        # Extract owner and repository name from the clone URL
        owner_and_repo = "/".join(clone_url.split('/')[-2:])

        # Make a GET request to get README information
        readme_url = readme_endpoint.format(owner_and_repo=owner_and_repo)
        response = requests.get(readme_url, headers=headers)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            readme_data = response.json()
            # Access the README content using base64 decoding
            readme_content = base64.b64decode(readme_data['content']).decode('utf-8')

            # Check for the presence of keywords
            present_keywords = [keyword for keyword in keywords_to_check if keyword in readme_content.lower()]

            # Print repository information to the console
            output_message = f"Repository: {repo_name}\nClone URL: {clone_url}\n"
            if present_keywords:
                output_message += f"Keywords Present: {', '.join(present_keywords)}\n"
            else:
                output_message += "No keywords found in README\n"
            output_message += '=' * 50 + '\n'
            
            print(output_message)
            
            # Write repository information to the output file
            output_file.write(output_message)
        elif response.status_code == 404:
            print(f"README not found for {repo_name}")
        else:
            print(f"Failed to retrieve README for {repo_name}: {response.text}")
